In [21]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()
import re

In [4]:
url = "https://ews.tropmet.res.in/"

In [5]:
await page.goto(url, wait_until='networkidle', timeout=0)
await page.locator("div.slideshow-container").wait_for()

In [6]:
await page.locator("div.slideshow-container strong").all_text_contents()

['Current AQI - 2672025-01-28 01',
 'Forecast AQI - 1702025-01-28 12',
 'Current AQI - 1052024-12-31 23',
 'Forecast AQI - 792025-01-27 19',
 'Current AQI - 1432025-01-22 09',
 'Forecast AQI - 2102025-01-26 19',
 'Current AQI - 1612023-12-31 23',
 'Forecast AQI - 3222025-01-27 19',
 'Current AQI - 1332025-01-23 14',
 'Forecast AQI -1312025-01-27 19',
 'Current AQI - 02025-01-23 14',
 'Forecast AQI - 422025-01-27 20',
 'Current AQI -2232025-01-22 09',
 'Forecast AQI -1212025-01-27 19',
 'Current AQI - 842025-01-22 09',
 'Forecast AQI - 2062025-01-27 19',
 'Current AQI - 622024-07-11 16',
 'Forecast AQI - 862025-01-26 19']

In [7]:
dates = await page.locator("div.slideshow-container strong").all_text_contents()

In [8]:
dates[2]

'Current AQI - 1052024-12-31 23'

In [9]:
date = [re.search(r'\d{4}-\d{2}-\d{2}', item).group() for item in dates]

In [10]:
date


['2025-01-28',
 '2025-01-28',
 '2024-12-31',
 '2025-01-28',
 '2025-01-22',
 '2025-01-26',
 '2023-12-31',
 '2025-01-27',
 '2025-01-23',
 '2025-01-27',
 '2025-01-23',
 '2025-01-28',
 '2025-01-22',
 '2025-01-27',
 '2025-01-22',
 '2025-01-27',
 '2024-07-11',
 '2025-01-26']

In [11]:
on_date = [elem for index, elem in enumerate(date) if (index + 1) % 2 != 0]
on_date

['2025-01-28',
 '2024-12-31',
 '2025-01-22',
 '2023-12-31',
 '2025-01-23',
 '2025-01-23',
 '2025-01-22',
 '2025-01-22',
 '2024-07-11']

In [12]:
result = await page.locator("text tspan.highcharts-text-outline").all_text_contents()

In [13]:
result

['267\u200b',
 '165\u200b',
 '105\u200b',
 '81\u200b',
 '143\u200b',
 '211\u200b',
 '161\u200b',
 '323\u200b',
 '133\u200b',
 '160\u200b',
 '0\u200b',
 '32\u200b',
 '223\u200b',
 '141\u200b',
 '84\u200b',
 '211\u200b',
 '62\u200b',
 '94\u200b']

In [14]:
All = [element.replace("\u200b", "") for element in result]
All

['267',
 '165',
 '105',
 '81',
 '143',
 '211',
 '161',
 '323',
 '133',
 '160',
 '0',
 '32',
 '223',
 '141',
 '84',
 '211',
 '62',
 '94']

In [15]:
AQI = [elem for index, elem in enumerate(All) if (index + 1) % 2 != 0]
AQI

['267', '105', '143', '161', '133', '0', '223', '84', '62']

In [16]:
city = ['Delhi', 'Pune', 'Mumbai', 'Kolkata', 'Bengalore', 'Lucknow', 'Patna', 'Hyderabad', 'Ahmedabad']

In [17]:
result_date_aqi = [{"AQI": aqi, "on_date": dt, "city": city} for aqi, dt, city in zip(AQI, on_date, city)]
result_date_aqi

[{'AQI': '267', 'on_date': '2025-01-28', 'city': 'Delhi'},
 {'AQI': '105', 'on_date': '2024-12-31', 'city': 'Pune'},
 {'AQI': '143', 'on_date': '2025-01-22', 'city': 'Mumbai'},
 {'AQI': '161', 'on_date': '2023-12-31', 'city': 'Kolkata'},
 {'AQI': '133', 'on_date': '2025-01-23', 'city': 'Bengalore'},
 {'AQI': '0', 'on_date': '2025-01-23', 'city': 'Lucknow'},
 {'AQI': '223', 'on_date': '2025-01-22', 'city': 'Patna'},
 {'AQI': '84', 'on_date': '2025-01-22', 'city': 'Hyderabad'},
 {'AQI': '62', 'on_date': '2024-07-11', 'city': 'Ahmedabad'}]

In [18]:
df = pd.DataFrame(result_date_aqi)

In [19]:
df.to_csv("major_cities_aqi.csv", index=False)

In [20]:
pd.read_csv("major_cities_aqi.csv")

,AQI,on_date,city
0,267,2025-01-28,Delhi
1,105,2024-12-31,Pune
2,143,2025-01-22,Mumbai
3,161,2023-12-31,Kolkata
4,133,2025-01-23,Bengalore
5,0,2025-01-23,Lucknow
6,223,2025-01-22,Patna
7,84,2025-01-22,Hyderabad
8,62,2024-07-11,Ahmedabad
